In [1]:
import csv
import operator
from typing import Annotated, List, Dict, Any, TypedDict
from langgraph.graph import StateGraph, END

# --- 1. 상태(State) 정의 ---
class AgentState(TypedDict):
    # 입력 데이터
    keywords: List[str]            # JSON에서 추출한 검색 키워드
    customer_name: str             # 고객 이름
    template: str                  # 채워야 할 템플릿 문장
    
    # 처리 과정 데이터
    matched_product: Dict[str, str] # DB에서 찾은 상품 정보
    
    # 최종 결과
    final_message: str

# --- 2. 노드(Node) 함수 정의 ---

def load_dummy_db(filename="products.csv"):
    """CSV 파일을 읽어서 리스트 딕셔너리로 반환하는 헬퍼 함수"""
    products = []
    # 테스트를 위해 파일이 없으면 즉석에서 데이터 생성
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                products.append(row)
    except FileNotFoundError:
        # 파일이 없을 경우 코드 내 더미 데이터 사용
        products = [
            {"product_name": "일리윤 세라마이드 아토 로션", "keywords": "겨울,보습,바디케어", "offer_text": "15% 할인 쿠폰", "deep_link": "app://P001"},
            {"product_name": "헤라 블랙 쿠션", "keywords": "메이크업,커버,여름", "offer_text": "퍼프 증정", "deep_link": "app://P002"}
        ]
    return products

def retrieve_product_node(state: AgentState) -> Dict[str, Any]:
    """
    키워드를 기반으로 가장 적합한 상품을 찾습니다.
    """
    print(f"\n[Search] 키워드 검색 시작: {state['keywords']}")
    
    db_products = load_dummy_db()
    target_keywords = set(state['keywords'])
    
    best_product = None
    max_overlap = -1
    
    for product in db_products:
        # CSV의 'keywords' 컬럼을 파싱
        prod_keywords = set(k.strip() for k in product['keywords'].split(','))
        
        # 교집합 개수 확인 (간단한 매칭 로직)
        overlap = len(target_keywords.intersection(prod_keywords))
        
        if overlap > max_overlap:
            max_overlap = overlap
            best_product = product
            
    if best_product:
        print(f"[Search] 상품 발견: {best_product['product_name']} (일치 키워드 수: {max_overlap})")
        return {"matched_product": best_product}
    else:
        print("[Search] 적절한 상품을 찾지 못했습니다.")
        # fallback 상품 설정
        return {"matched_product": db_products[0]}

def generate_message_node(state: AgentState) -> Dict[str, Any]:
    """
    찾은 상품 정보로 템플릿 슬롯을 채웁니다.
    """
    print("\n[Gen] 메시지 생성 중...")
    
    product = state['matched_product']
    template = state['template']
    
    # 슬롯 매핑 데이터 준비
    slot_values = {
        "customer_name": state['customer_name'],
        "product_name": product['product_name'],
        "offer": product['offer_text'],
        "cta": product['deep_link']
    }
    
    # 파이썬 f-string 스타일 포매팅 사용 (Template 변수 치환)
    try:
        completed_message = template.format(**slot_values)
    except KeyError as e:
        completed_message = f"에러: 템플릿에 필요한 키가 없습니다 -> {e}"
        
    return {"final_message": completed_message}

# --- 3. 그래프(Graph) 구성 ---

workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("retrieve_product", retrieve_product_node)
workflow.add_node("generate_message", generate_message_node)

# 엣지 연결 (순차 실행)
workflow.set_entry_point("retrieve_product")
workflow.add_edge("retrieve_product", "generate_message")
workflow.add_edge("generate_message", END)

# 컴파일
app = workflow.compile()

# --- 4. 실행 테스트 ---

# User가 제공한 JSON 컨텍스트 가정
input_data = {
    # JSON에서 추출한 키워드
    "keywords": ["겨울", "보습", "재구매", "친근한", "루틴"], 
    
    # User Profile 정보 (가정)
    "customer_name": "김아모레",
    
    # JSON의 body_with_slots 내용
    "template": "고객님, 겨울철 보습 루틴을 고민 중이시라면 {product_name}이 좋을 것 같아요. 지금 앱에서 확인해 보세요!\n{customer_name}님을 위한 혜택: {offer}\n바로가기: {cta}",
    
    "matched_product": {},
    "final_message": ""
}

# 실행
result = app.invoke(input_data)

# 결과 출력
print("-" * 30)
print(">>> 최종 완성된 메시지 Result:")
print(result["final_message"])
print("-" * 30)


[Search] 키워드 검색 시작: ['겨울', '보습', '재구매', '친근한', '루틴']
[Search] 상품 발견: 일리윤 세라마이드 아토 로션 (일치 키워드 수: 2)

[Gen] 메시지 생성 중...
------------------------------
>>> 최종 완성된 메시지 Result:
고객님, 겨울철 보습 루틴을 고민 중이시라면 일리윤 세라마이드 아토 로션이 좋을 것 같아요. 지금 앱에서 확인해 보세요!
김아모레님을 위한 혜택: 15% 할인 쿠폰 증정
바로가기: amoremall://product/P001
------------------------------
